# Evaluating the Models

## This Juypter Notebook was created to easily load the models and set/change the directories to test the models, all in one place.

In [1]:
# Allows evaluate.ipynb to access modules in "sibling" directories
import sys
sys.path.append("./") # Run this from parent directory

In [1]:
import os
from database import Database
from models.altered_xception import AlteredXception
from models.faster_r_cnn import FasterRCNN
from models.rmac_model import RMACModel
from utils import grab_images_and_paths, grab_all_image_paths, resize_images, DATABASE_PATH
from models.query import query_image

/Users/lchris/.pyenv/versions/3.9.1/lib/python3.9/site-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.9.1` and `torch==1.8.1` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


## Loading The Models and Data
> Note: if you get any error surrounding a missing layer, a simple restart of the kernel should fix it.

In [2]:
# NOTE: if the RMAC model needs to get descriptions of many different square regions, it will take a long time to load/initialize (roughly a minute).
ax_model = AlteredXception()
rmac_model = RMACModel(ax_model.model.get_layer("conv2d_3").output_shape[1:], 3)
rcnn_model = FasterRCNN()

2022-12-07 13:31:34.639008: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/lchris/Desktop/Coding/schoolprojects/comp490/COMPS/models/rmac_model.py:120: RuntimeWarning: invalid value encountered in double_scalars
  b = (W - wl) / (l + Wd - 1)
/Users/lchris/Desktop/Coding/schoolprojects/comp490/COMPS/models/rmac_model.py:125: RuntimeWarning: invalid value encountered in double_scalars
  b = (H-wl)/(l+Hd-1)


In [3]:
# Choose what models to evaluate, by including or discluding them from this array
models = [ax_model, rmac_model, rcnn_model]

In [4]:
# test_dir is the directory where our "database" of test images/frames and query images should exist. 
test_dir = "/Users/lchris/Desktop/Coding/schoolprojects/comp490/COMPS/eval/test_images/test_games"
frame_paths, frames = grab_images_and_paths(os.path.join(test_dir, "frames"), num_images=1)
query_image_paths = grab_all_image_paths(os.path.join(test_dir, "query_images"))
query_image_paths

['/Users/lchris/Desktop/Coding/schoolprojects/comp490/COMPS/eval/test_images/test_games/query_images/mw3_frame.jpg',
 '/Users/lchris/Desktop/Coding/schoolprojects/comp490/COMPS/eval/test_images/test_games/query_images/mw2_clickbait.jpeg',
 '/Users/lchris/Desktop/Coding/schoolprojects/comp490/COMPS/eval/test_images/test_games/query_images/mw2query.jpeg',
 '/Users/lchris/Desktop/Coding/schoolprojects/comp490/COMPS/eval/test_images/test_games/query_images/mw2_distorted.jpg']

In [5]:
# Get the YouTube video "ground truth" from the IDs we picked, stored in a .txt file
ground_truth_ids = []
ground_truth_file_path = os.path.join(test_dir, "ground_truth.txt")
with open(ground_truth_file_path, "r") as ground_truth:
    for line in ground_truth.readlines():
        youtube_ground_truth_id = line.split("/")[-1].split(".")[0]
        ground_truth_ids.append(youtube_ground_truth_id)
ground_truth_ids

['-tTaYmNB-IM',
 'q_hZHqNnwAU',
 'r3FaHBe1rPw',
 'S7Nqx_pyEl0',
 'cmh9FnLbE5s',
 'huGVGe3Afng',
 'kQnQrYmhXW8',
 '_lAtb14VGm8',
 '0J57bBrDDLc',
 '5X6SctBSd9A']

In [6]:
# Initialize models to query from the evaluation database
database = Database(os.path.join(test_dir, "predictions")) # Small DB
# database = Database(DATABASE_PATH) # Large DB 
for model in models:
    model.database = database

In [7]:
# Gathering predictions for each of the frames in the dataset, for each model
if not os.path.exists(os.path.join(test_dir, "predictions")):
    # Make the folder to store our representations
    os.mkdir(os.path.join(test_dir, "predictions"))
for model in models:
    print("Current model: ", model)
    if (isinstance(model, AlteredXception)) and len(database.prediction_image_paths) < len(frame_paths):
        model_predictions = model.predict_images(frames)
        database.store_predictions(model_predictions, frame_paths)
    if (isinstance(model, RMACModel)) and len(database.rmac_prediction_image_paths) < len(frame_paths):
        model_predictions = model.predict_images(frames)
        database.store_rmac_predictions(model_predictions, frame_paths)
    if (isinstance(model, FasterRCNN)) and len(database.object_predictions) < len(frame_paths):
        every_10_frame_paths = frame_paths[0::10]
        model_predictions = model.predict_image_paths(every_10_frame_paths)
        database.store_object_data(model_predictions, every_10_frame_paths)

Current model:  <models.altered_xception.AlteredXception object at 0x107c268e0>
Current model:  <models.rmac_model.RMACModel object at 0x16d64a610>
Current model:  <models.faster_r_cnn.FasterRCNN object at 0x16d6e35e0>


In [8]:
def post_process_predictions(predictions):
    model_preds = {}
    for model in predictions:
        video_id_preds = []
        for video_link in predictions[model].keys():
            video_id = video_link.split("/")[-1] # We only want the last part of the path
            video_id_preds.append(video_id)
        model_preds[model] = video_id_preds
    return model_preds

In [9]:
results = {}
for query_image_path in query_image_paths:
    results[query_image_path] = {}
    results[query_image_path] = post_process_predictions(query_image(query_image_path, models, len(ground_truth_ids)))
# results = query_image(query_image_paths[0], models, len(ground_truth_ids))
results

1/1 [==============================] - 0s 51ms/step


{'/Users/lchris/Desktop/Coding/schoolprojects/comp490/COMPS/eval/test_images/test_games/query_images/mw3_frame.jpg': {'CNN': ['q_hZHqNnwAU',
   '4Y7IB_gTCs4',
   'kQnQrYmhXW8',
   'f1gNEJh6eyg',
   '0VgTeTpv48E',
   '50DkxNJXAOY',
   '0J57bBrDDLc',
   'Q38a-Vt_p2E',
   'bsNUaZf9oyg',
   'rebo0qHivSE'],
  'CNN + RMAC': ['q_hZHqNnwAU',
   'lcxxGZqKj8s',
   'U_YEdWlv5N4',
   'bsNUaZf9oyg',
   'vqyOxkyswVQ',
   'I8uehX-eXoc',
   'juyUhncX9jk',
   'RMRuHrr7NrQ',
   'bBIpqffkIm0',
   '3rGvfEQ4QtQ'],
  'RCNN': ['S7Nqx_pyEl0',
   'NehypH0u4XY',
   'rebo0qHivSE',
   '5D4T7GPhQOM',
   'f1gNEJh6eyg',
   'WinRtwzZaLo',
   'RvWx16ItqM8',
   'Q38a-Vt_p2E',
   'J35d7bA4hCE',
   'RMRuHrr7NrQ']},
 '/Users/lchris/Desktop/Coding/schoolprojects/comp490/COMPS/eval/test_images/test_games/query_images/mw2_clickbait.jpeg': {'CNN': ['huGVGe3Afng',
   'WinRtwzZaLo',
   '-tTaYmNB-IM',
   'S7Nqx_pyEl0',
   'HNPdE3g4jJ4',
   'RYpNYKWhyQ8',
   'AmHJwCJTVZc',
   'm00Bc2ePbYY',
   'NehypH0u4XY',
   '4Y7IB_gTCs4'],
  

In [10]:
matches = {}
for query_image_path in query_image_paths:
    curr_results = results[query_image_path]
    matches[query_image_path] = {}
    for model in results[query_image_path]:
        num_matches = []
        for index, id in enumerate(ground_truth_ids):
            # print(model, id, results[query_image_path][model])
            if id in results[query_image_path][model]:
                num_matches.append(index+1)
        matches[query_image_path][model] = num_matches
matches

{'/Users/lchris/Desktop/Coding/schoolprojects/comp490/COMPS/eval/test_images/test_games/query_images/mw3_frame.jpg': {'CNN': [2,
   7,
   9],
  'CNN + RMAC': [2],
  'RCNN': [4]},
 '/Users/lchris/Desktop/Coding/schoolprojects/comp490/COMPS/eval/test_images/test_games/query_images/mw2_clickbait.jpeg': {'CNN': [1,
   4,
   6],
  'CNN + RMAC': [1, 6],
  'RCNN': [5, 7]},
 '/Users/lchris/Desktop/Coding/schoolprojects/comp490/COMPS/eval/test_images/test_games/query_images/mw2query.jpeg': {'CNN': [1,
   4,
   6,
   9],
  'CNN + RMAC': [6, 9],
  'RCNN': [3, 5, 9]},
 '/Users/lchris/Desktop/Coding/schoolprojects/comp490/COMPS/eval/test_images/test_games/query_images/mw2_distorted.jpg': {'CNN': [4,
   6,
   9],
  'CNN + RMAC': [6],
  'RCNN': []}}